In [1]:
############################# Importing all req. libreries ############################################
from pymongo import MongoClient
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
import ipywidgets as ipw
client = MongoClient("mongodb://localhost:27017")
database = client["Open_Source_DB"]
collection = database["Location_Details"]
#collection.createIndex( { location : "2dsphere" } )
    
i_lat =ipw.FloatText(value=7.5,description='Latitude:',disabled=False)
i_lon =ipw.FloatText(value=7.5,description='Longitude:',disabled=False)

ipw.HBox([i_lat , i_lon ])

In [2]:
print( i_lat.value  , '--------------' , i_lon.value )

48.775845 -------------- 9.182932


In [3]:
############################# MongoDB Pipeline  ############################################

pipeline = [
    {
        u"$geoNear": {
            u"near": {
                u"type": u"Point",
                u"coordinates": [
                    i_lon.value,
                    i_lat.value
                ]
            },
            u"distanceField": u"dist.calculated",
            u"maxDistance": 100000
        }
    }, 
    {
        u"$project": {
            u"Entity": u"$Type of User",
            u"coordinates": u"$location.coordinates",
            u"_id": 0.0
        }
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = False
)


lats = []
lons = []
ty = []
try:
    for doc in cursor:
        print(doc)
        lats.append(doc["coordinates"][1])
        lons.append(doc["coordinates"][0])
        ty.append(doc["Entity"])
finally:
    client.close()


{'Entity': 'End User', 'coordinates': [9.180819, 48.782343]}
{'Entity': 'End User', 'coordinates': [9.316550999999999, 48.7374]}
{'Entity': 'End User', 'coordinates': [9.315456, 48.831538]}
{'Entity': 'End User', 'coordinates': [9.195419000000001, 48.893886]}
{'Entity': 'End User', 'coordinates': [9.029552, 48.684953]}
{'Entity': 'End User', 'coordinates': [9.05, 48.533333]}
{'Entity': 'End User', 'coordinates': [9.213697999999999, 48.488779]}
{'Entity': 'End User', 'coordinates': [8.737612, 48.716828]}
{'Entity': 'End User', 'coordinates': [9.656463, 48.703582]}
{'Entity': 'End User', 'coordinates': [8.698922999999999, 48.884361]}
{'Entity': 'End User', 'coordinates': [9.220541, 49.13995]}
{'Entity': 'End User', 'coordinates': [9.734167, 49.105833000000004]}
{'Entity': 'Retailer', 'coordinates': [8.852117999999999, 48.277142]}
{'Entity': 'End User', 'coordinates': [8.40444, 49.009365]}
{'Entity': 'End User', 'coordinates': [9.151062, 49.353573]}
{'Entity': 'End User', 'coordinates': [

In [4]:
####################### Visulizing the results in Map ###################################################
m = Basemap(projection='mill',
                resolution='l') 

m.drawcoastlines()
m.drawcountries(linewidth=0.1)
m.drawstates()

i = 0
for lat , lon in zip(lats,lons):
    x,y = m.projtran(lon,lat)
    if ty[i] == 'End User' : m.plot(x,y,'g*',markersize=10)
    if ty[i] == 'Retailer' : m.plot(x,y,'y*',markersize=10)
    if ty[i] == 'Supplier' : m.plot(x,y,'b*',markersize=10)
    if ty[i] == 'Manufacturer' : m.plot(x,y,'r*',markersize=10)
    i = i+1
plt.show()

C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:3: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:6: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  
C:\Users\anilk\Anaconda3\envs\Final\lib\site-packages\ipykernel_launcher.py:7: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  import sys
